# Date solved: 17 November 2018

# Problem
https://projecteuler.net/problem=81

In the 5 by 5 matrix below, the minimal path sum from the top left to the bottom right, by only moving to the right and down, is indicated in bold red and is equal to 2427.

$$
\begin{pmatrix}
\color{red}{131} & 673 & 234 & 103 & 18\\
\color{red}{201} & \color{red}{96} & \color{red}{342} & 965 & 150\\
630 & 803 & \color{red}{746} & \color{red}{422} & 111\\
537 & 699 & 497 & \color{red}{121} & 956\\
805 & 732 & 524 & \color{red}{37} & \color{red}{331}
\end{pmatrix}
$$

Find the minimal path sum, in matrix.txt (right click and "Save Link/Target As..."), a 31K text file containing a 80 by 80 matrix, from the top left to the bottom right by only moving right and down.

# Solution
We can use a shortest path algorithm to find the minimal path from top left (source) to bottom right (target). In order to do so, we need to formulate the matrix as a directed graph. Let's take a small matrix as an easy example to construct a directed graph:

$$
\begin{pmatrix}
100 & 200 & 300 \\
50 & 100 & 150 \\
100 & 150 & 200 \\
\end{pmatrix}
$$

## Construction of a directed graph 
Let $G$ be a directed graph. The number of nodes is equal to the number of elements in the matrix + 1, where the + 1 is necessary to include the source node. Moreover, we denote the nodes with tuples $\{(s,0), (0,0), (0,1), (0,2), \dots, (2,1), (2,2)\}$, where each tuple $(i,j)$ corresponds to the $i+1,j+1$-th element of the matrix and $(s,0)$ is the source node. Mathematically speaking, the top-left element in the matrix is $a_{11}$, but since Python starts indexing with 0, this indexing is more convenient. Hence, the node $(0,0)$ corresponds to the matrix element $a_{11}$. 

Next, we construct the edges of $G$. Note that we only construct directed edges in this example. First, observe that we can only move to the right and down in the matrix, so we need to construct an edge from element to its right neighbour and its bottom neighbour. Hence, for each node $(i, j)$, we construct an edge from $(i, j)$ to $(i, j+1)$ (right neighbour) and a node from $(i,j)$ to $(i+1,j)$ (bottom neighbour). The length of the edge ($(i,j)$, $(k,l)$) corresponds to the matrix value of $(k,l)$. Furthermore, we add just one edge from node $(s,0)$ to node $(0,0)$ with the length of the matrix elements $a_{11}$.

For example, node $(0,0)$ will have an edge to node $(0,1)$ with length $200$ and an edge to node $(1,0)$ with length $50$. 

Note that we only construct an edge if the neighbour exists (so for node $(2,2)$, there exists no right nor bottom neighbour and hence we do not add any edges.)

This completes the graph construction. 

## Implementation using NetworkX
We construct the directed graph using the NetworkX package in Python. This package offers many graph data structures and algorithms that will come in handy for this problem. Moreover, the numpy package will be useful to store the matrix.

In [48]:
import networkx as nx
import numpy as np

We start by importing the text file that contains the matrix. It is a plain text file, where each row corresponds to the rows of the matrix. The elements are seperated by commas. 

In [31]:
mat = np.array([x.split(',') 
                for x in [l.rstrip() 
                          for l in open("pe81.txt").readlines()]], 
               dtype='int32')
print(mat)

[[4445 2697 5115 ... 2758 3748 5870]
 [1096   20 1318 ... 4187 9353 9377]
 [9607 7385  521 ... 9515 6385 9230]
 ...
 [2265 8192 1763 ... 7456 5128 5294]
 [2132 8992 8160 ... 5634 1113 5789]
 [5304 5499  564 ... 2751 3406 7981]]


Next, we define a function to construct a directed graph from a numpy matrix (according to the path movement rules in this particular problem). 

In [43]:
def mat_to_DiGraph(M):
    """Constructs a directed graph (for Problem 81) from a numpy matrix. """
    G = nx.DiGraph()
    n = M.shape[0] # number of rows
    m = M.shape[1] # number of cols
    
    # Construct initial edge from (s,0) to (1,1)
    G.add_edge(('s',0),(0,0),weight=M[0,0])
    
    # Construct the edges for other nodes
    for i in range(0, n):
        for j in range(0, m):
            # Add right and bottom neighbour
            if i < n-1 and j < m-1:
                G.add_weighted_edges_from([((i,j),(i,j+1),M[i,j+1]),
                                           ((i,j),(i+1,j),M[i+1,j])])
            
            # Only add right neighbour
            elif j < m - 1:
                G.add_weighted_edges_from([((i,j),(i,j+1),M[i,j+1])])
            
            # Only add bottom neighbour
            elif i < n - 1:
                G.add_weighted_edges_from([((i,j),(i+1,j),M[i+1,j])])
    
    return(G)

The final step is to apply the shortest path algorithm. For this, we use Dijkstra's shortest path algorithm, which is also included in the NetworkX package. 

In [47]:
def minPathSum(M):
    """Finds the minimum path sum from problem 81."""
    G = mat_to_DiGraph(M)
    n, m = [M.shape[0],M.shape[1]]
    return(nx.dijkstra_path_length(G, ('s',0), (n-1,m-1)))

print(minPathSum(mat))

427337
